# EMP-1

Importando las librerias necesarias para el proyecto

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import requests



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.3/102.3 kB 10.7 MB/s eta 0:00:00


In [ ]:
#Instalando Folium
!pip install folium

Solicitud HTTP de la pagina web

In [ ]:
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=995657573'
table = pd.read_html(url)[0]

Renombrando las columnas

In [ ]:
table.columns = ['PostalCode', 'Borough', 'Neighborhood']


Eliminar las filas donde se tenga el registro "Not assigned" en la columna "Borough"

In [ ]:
table = table[table['Borough'] != 'Not assigned']

Combinacion de las filas con los codigos postales 

In [ ]:
table = table.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
table['Neighborhood'] = np.where(table['Neighborhood'] == 'Not assigned', table['Borough'], table['Neighborhood'])


Mostrando los resultados en el marco de datos

In [4]:
display(table)




,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


# EMP-2

Datos de geolocalizacion:

In [6]:
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=995657573'
table = pd.read_html(url)[0]

# Renombrar las columnas
table.columns = ['PostalCode', 'Borough', 'Neighborhood']

# Eliminar las filas donde la columna "Borough" es "Not assigned"
table = table[table['Borough'] != 'Not assigned']

# Combinar las filas con códigos postales repetidos en una sola fila separando los vecindarios con comas
table = table.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
table.rename(columns={'PostalCode': 'Postal Code'}, inplace=True)

# Si una celda tiene un municipio, pero un vecindario No asignado, entonces el vecindario será el mismo que el municipio.
table['Neighborhood'] = np.where(table['Neighborhood'] == 'Not assigned', table['Borough'], table['Neighborhood'])

# Leer el archivo csv con las coordenadas geográficas
shortened_url = "http://cocl.us/Geospatial_data"

response = requests.get(shortened_url)
actual_url = response.url
geo_data = pd.read_csv(actual_url)

# Combinar los datos de la tabla y las coordenadas geográficas
merged_table = pd.merge(table, geo_data, on='Postal Code')

# Reordenar las columnas para que tengan el mismo orden que se requiere en el enunciado
final_table = merged_table[['Postal Code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]

# Mostrar el número de filas del marco de datos resultante
display(final_table)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


In [7]:
import folium

# Crear un mapa centrado en Toronto
map_toronto = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

# Agregar un marcador para cada vecindario
for lat, lng, borough, neighborhood in zip(final_table['Latitude'], final_table['Longitude'], final_table['Borough'], final_table['Neighborhood']):
    label = f"{neighborhood}, {borough}"
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

# Mostrar el mapa
map_toronto